In [0]:
exec(open('./helpers.py').read())
class T():
    def newest(self, qry, prt, tbl='', sel=''):
        """The OPEIR daily snapshot experienced occasional glitched causing incomplete copies.
        Consequently, records can have a "gap" where they vanish then reappear later. This function fixes this issue."""
        prt = join(prt, ', ')
        if tbl == '':
            tbl = qry
        if sel != '':
            sel = ','+join(sel)

        qry = f"""
select
    {prt}
    ,current_date
    ,min(current_date) over (partition by {prt}) as first_date --first date this record appeared
    ,max(current_date) over (partition by {prt}) as last_date  --last  date this record appeared
    ,least(timestamp('{self.date}'), max(current_date) over ()) as cutoff_date  --cutoff_date is the ealier of self.date and last date of ANY record
from
    {qry.strip()}
qualify
    cutoff_date between first_date and dateadd(last_date, 5)  -- keep records where cutoff_date falls between that record's first & last appearance (+5 days margin handles rare technical glitch where a record for this upcoming term is missing TODAY but might reappear soon ... does not apply for prior terms).
"""

        qry = f"""
select
    *
from {subqry(qry)}
where
    current_date <= cutoff_date  -- discard records after cutoff_date
qualify
    row_number() over (partition by {prt} order by current_date desc) = 1  -- keep most recent remaining record
"""

        qry = f"""
select distinct
    pidm
    ,first_date
    ,current_date
    ,last_date
    ,{get_desc('term_code')}
    ,{get_desc('levl_code')}
    ,{get_desc('styp_code')}
    ,{get_desc('camp_code')}
    ,{get_desc('coll_code_1')}
    ,{get_desc('dept_code')}
    ,{get_desc('majr_code_1')}
    --,gender
    ,spbpers_sex as gender
    ,birth_date
    ,{get_desc('spbpers_lgcy_code')}
    ,gorvisa_vtyp_code is not null as international
    ,gorvisa_natn_code_issue as natn_code, (select stvnatn_nation from {catalog}saturnstvnatn where gorvisa_natn_code_issue = stvnatn_nation limit 1) as natn_desc
    ,{coalesce('race_asian')}
    ,{coalesce('race_black')}
    ,coalesce(spbpers_ethn_cde=2, False) as race_hispanic
    ,{coalesce('race_native')}
    ,{coalesce('race_pacific')}
    ,{coalesce('race_white')}
    {indent(sel)}
from {subqry(qry)} as A

left join
    {tbl}
using
    ({prt}, current_date)

left join
    {catalog}spbpers_v
on
    pidm = spbpers_pidm

left join (
    select
        *
    from
        {catalog}generalgorvisa
    qualify
        row_number() over (partition by gorvisa_pidm order by gorvisa_seq_no desc) = 1
    )
on
    pidm = gorvisa_pidm

left join (
    select
        gorprac_pidm
        ,max(gorprac_race_cde='AS') as race_asian
        ,max(gorprac_race_cde='BL') as race_black
        ,max(gorprac_race_cde='IN') as race_native
        ,max(gorprac_race_cde='HA') as race_pacific
        ,max(gorprac_race_cde='WH') as race_white
    from
        {catalog}generalgorprac
    group by
        gorprac_pidm
    )
on
    pidm = gorprac_pidm
"""
        return qry


    def get_registrations(self, show=False):
        def fcn():
            tbl = f'dev.opeir.registration_{self.term_desc}_v'
            if spark.catalog.tableExists(tbl):
                qry = self.newest(
                    tbl = tbl,
                    prt = ['pidm','crn'],
                    sel = ['credit_hr as count', 'subj_code || crse_numb as crse_code'],
                    qry = f"""
    {tbl} as A
where
    credit_hr > 0
    and subj_code <> 'INST'""")
            return run_qry(qry, show)
        #         B = A.groupmy(['pidm','crse_code'])[['count']].sum().reset_index('crse_code') #combine if enrolled in >1 crn for same crse_code (rare)
        #         C = B.groupmy('pidm')[['count']].sum() #compute total sch
        #         D = C.copy()
        #         B['count'] = 1 #crse_code headcount
        #         D['count'] = 1 #overall headcount
        #         E = pd.concat([B, C.assign(crse_code='_tot_sch'), D.assign(crse_code='_headcnt'), D.assign(crse_code='_proba')])
        #         F = A.drop(columns=['count','crse_code']).sort_values('current_date').groupmy('pidm').last()
        #         df = F.join(E)
        #     else:
        #         # placeholder if table DNE
        #         df = pd.DataFrame(columns=['pidm','levl_code','styp_code','count','crse_code']).set_index('pidm')
        #     get_duplicates(df, ['pidm','crse_code'])
        #     return df
        # return self.run(fcn, f'registrations/{self.date}/{self.term_code}', root=current)[0]
t = T()